In [52]:
import tensorflow as tf

In [53]:
# Creating random DataSet for general Sales number

daily_sales_numbers = [23,15,-120, 21, 55, 32,98,12]

In [54]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [55]:
for items in tf_dataset.as_numpy_iterator():
#for items in tf_dataset:
    print(items)
    # You can also convert these objects into number by using Numpy library
    #print(items.numpy())

23
15
-120
21
55
32
98
12


In [56]:
# taking 1st three elements from dataset 

for items in tf_dataset.take(3):
    print(items.numpy())

23
15
-120


In [57]:
# Filter out -ve data points or correcting the negative values

tf_dataset = tf_dataset.filter(lambda x:x>0)
for items in tf_dataset.as_numpy_iterator():
    print(items)

23
15
21
55
32
98
12


In [58]:
# For examples the above sales prices are in Dollars we want to convert these numbers into PKR
# $1 = 288 pkr

tf_dataset = tf_dataset.map(lambda x: x*288)
for items in tf_dataset.as_numpy_iterator():
    print(items)

6624
4320
6048
15840
9216
28224
3456


In [59]:
# For randomly shuffling dataset , especially it will for image dataset

tf_dataset = tf_dataset.shuffle(3)
for items in tf_dataset.as_numpy_iterator():
    print(items)

6624
4320
15840
28224
6048
9216
3456


In [60]:
# Batching the dataset

for items_batch in tf_dataset.batch(4):
    print(items_batch.numpy())

[ 6624 15840  4320 28224]
[9216 6048 3456]


In [61]:
# Doing the above things in one line

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda y: y*288).shuffle(3).batch(2)

for items in tf_dataset.as_numpy_iterator():
    print(items)

[ 4320 15840]
[6624 6048]
[28224  3456]
[9216]


In [62]:
# Building Tensorflow pipeLine

images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

for files in images_ds.take(5):
    print(files)

tf.Tensor(b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\A cat appears to have caught the....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\Adopt-A-Cat Month\xc2\xae - American Humane....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\All About Your Cat_s Tongue.jpg', shape=(), dtype=string)


In [63]:
# Shuffle Dataset

images_ds = images_ds.shuffle(200)

for files in images_ds.take(5):
    print(files.numpy())

b'images\\dog\\The US Army is testing augmented....jpg'
b'images\\dog\\Subaru Shows Love for Dogs Through the....jpg'
b'images\\dog\\2nd pet dog tests positive for COVID-19....jpg'
b'images\\dog\\Cats and Dogs.jpg'
b'images\\dog\\Trained dogs can smell coronavirus in....jpg'


In [64]:
# Classes name

classes_name = ['cat','dog']


In [65]:
# For training purpose we have to split the images, in TF we use take function for this purpose

image_count = len(images_ds)
image_count

130

In [66]:
# Now dividing the dataset in to tarining 80% and testing 20%

train_size = int(image_count*0.8)

train_ds = images_ds.take(train_size) # It will take 80% of dataset and rest skip the 20% of dataset
test_ds = images_ds.skip(train_size)

In [67]:
len(train_ds)

104

In [68]:
len(test_ds)

26

In [69]:
# In the  image path we already have class label, now we have to retrive that label from string

s = 'images\\dog\\Science_ Talking to Your Dog Means You....jpg'

s.split("\\")[-2]



'dog'

In [70]:
def get_label(file_path):
    import os 
    return tf.strings.split(file_path, os.path.sep)[-2]

In [71]:
# To get X_train part

def process_image(file_path):
    label = get_label(file_path)
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    
    return img, label

In [72]:
for t in train_ds.take(4):
    print(t.numpy())

b'images\\dog\\Rescue turns dog with untreatable tumor....jpg'
b'images\\cat\\Cat Advice _ Collecting a Urine Sample....jpg'
b'images\\dog\\Maltese Dog Breed Information_ Pictures....jpg'
b'images\\dog\\10 Teacup Dog Breeds for Tiny Canine Lovers.jpg'


In [73]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print("Image: ",img)
    print("Label: ",label)

Image:  tf.Tensor(
[[[178.7539   185.7539   143.7539  ]
  [185.875    195.875    161.03125 ]
  [185.9375   195.9375   160.98828 ]
  ...
  [194.875    199.46875  170.6875  ]
  [193.01953  197.01953  172.01953 ]
  [189.5039   195.       162.34766 ]]

 [[172.03125  184.28125  140.65625 ]
  [189.15625  200.       170.46875 ]
  [180.96484  190.96484  156.96484 ]
  ...
  [192.6289   197.22266  168.4414  ]
  [195.0586   199.0586   174.0586  ]
  [186.71875  195.71875  167.96875 ]]

 [[176.38672  187.01172  148.13672 ]
  [185.6836   196.52734  166.9961  ]
  [180.375    191.03516  155.1914  ]
  ...
  [195.15234  199.15234  172.66016 ]
  [195.47266  199.47266  176.47266 ]
  [188.10547  196.10547  173.10547 ]]

 ...

 [[160.53516  178.71484   97.796875]
  [145.14062  153.9336    98.49609 ]
  [144.71484  167.625     96.85156 ]
  ...
  [176.8164   186.72656  137.9336  ]
  [170.52734  175.83984  125.453125]
  [180.66406  188.3125   146.8164  ]]

 [[187.2539   198.08594  133.66797 ]
  [170.67969  189.

In [74]:
# Scale images

def scale(image, label):
    return image/255, label

In [75]:
train_ds = train_ds.map(scale)

for image, label in train_ds.take(5):
    print("*****Image: ", image.numpy()[0][0])
    print("*****Label: ", label.numpy())

*****Image:  [0.14411765 0.14019608 0.1637255 ]
*****Label:  b'dog'
*****Image:  [0.85505515 0.85505515 0.85505515]
*****Label:  b'cat'
*****Image:  [0.99607843 0.99607843 0.99607843]
*****Label:  b'dog'
*****Image:  [0.6349581 0.6433053 0.6313726]
*****Label:  b'dog'
*****Image:  [0.627451   0.5176471  0.42352942]
*****Label:  b'cat'
